In [0]:
%pip install pymongo[srv] torch transformers vaderSentiment

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pyspark
#import torch
import pymongo
from pyspark.sql import Row
from transformers import pipeline
from pyspark.sql.functions import col, udf, size, concat_ws, collect_list
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, FloatType
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import transformers

In [0]:
username = "admin"
password = "goal-diggers"
database = "reviewdb"
collection_name = "amazon"
ip_address = "msds697-goal-diggers.5f8jw.mongodb.net"

client = pymongo.MongoClient(f"mongodb+srv://{username}:{password}@{ip_address}")
db = client[database]
amazon_collection = db[collection_name]
amazon_collection.find_one()

Out[2]: {'_id': ObjectId('63fc65edf6b47c884faf1386'),
 'ASIN': 'B00015VKT4',
 'reviews': [{'review': "My first review but I have to say......About 15 years ago I developed an allergy to nickle (earrings, necklaces, belt buckles, buttons, bracelets!) and stopped wearing most jewelry. I have to tape over my belt buckles or any metal that contacts my skin and was diagnosed as severely allergic by my doc after allergy skin test. About a year ago I got my ears repierced with my daughter so we could have fun wearing jewelry and have tried every kind of hypo allergenic earring I could find: Studex, Sterling Silver 925, Surgical Steel, 14k gold, and had allergic reactions to all of them if worn for even an hour. Studex that my ears were pierced with was the worst! Bleeding, red itchy swollen earlobes a few weeks to heal in between each time I wore earrings! Searched Amazon for an hour to find these and did some research. They work as promised! I had to force them through (holes had partially c

In [0]:
##### Summarize the good and bad reviews separately

In [0]:
#Andres
# Initialize the SentimentIntensityAnalyzer object: {"$addFields": {"reviews.score": sia.polarity_scores("$reviews.review")}},
sia = SentimentIntensityAnalyzer()

# Define the pipeline
aggregation = [
    {"$unwind": "$reviews"},
    {"$addFields": {"rating":"$reviews.rating","title": "$reviews.title","review": "$reviews.review"}},
    {"$project": {"_id":1,"ASIN":1,"rating":1,"title": 1,"review":1}}
]

# Execute the pipeline
results = amazon_collection.aggregate(aggregation)

rows = [Row(ASIN=d['ASIN'], rating=d['rating'], review_text=d['title']+ " : " + d['review']) for d in results]

schema = StructType([
    StructField("ASIN", StringType(), False),
    StructField("rating", FloatType(), False),
    StructField("review", StringType(), False)
])
# create a DataFrame from the list of Rows
combined_text_df = spark.createDataFrame(rows, schema)
combined_text_df.cache()
# show the DataFrame
combined_text_df.show(1, truncate=False, vertical=True)


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
### Only use popular products with at least 30 reviews
print("There are",combined_text_df.distinct().count(),"products in our data")
popular_products = combined_text_df.groupBy("ASIN").count().filter(col("count") > 30)
popular_products_df = popular_products.join(combined_text_df, "ASIN", "inner").select("ASIN","rating","review")
print("There are",popular_products.count(), "products with at least 30 reviews, with a total of",popular_products_df.count(),"reviews")
popular_products_df.show(10, truncate=100)

There are 201959 products in our data
There are 1887 products with at least 30 reviews, with a total of 241661 reviews
+----------+------+----------------------------------------------------------------------------------------------------+
|      ASIN|rating|                                                                                              review|
+----------+------+----------------------------------------------------------------------------------------------------+
|B008RF8XFY|   1.0|One Star : Into the garbage within a couple minutes of receipt.  Way too short to do anything wit...|
|B008RF8XFY|   1.0|I am so disappointed! How the heck do u put in on : I am so disappointed! How the heck do u put i...|
|B008RF8XFY|   1.0|Size : It would be fine for a child...It is to small for an adult's wrist. I am a petite person a...|
|B008RF8XFY|   1.0|Not satisfied : I got the string  but it was way too small  for my vBulletin wrist. Managed to pu...|
|B008RF8XFY|   1.0|Completely came

In [0]:
# Andres 
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = sia.polarity_scores(text)
    if scores['compound'] > 0.05:
        return 'positive'
    if scores['compound'] < -0.05:
        return 'negative'
    return 'neutral'


sentiment_df = popular_products_df.withColumn("sentiment", udf(get_sentiment)(col("review")))
sentiment_df.cache()
sentiment_df.show(10, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ASIN      | B008RF8XFY                                                                                                                                                                                                                                                                                                                                                                                                                              

In [0]:
# Group all of the product's positive reviews together, negative reviews together and neutral reviews together.
review_df = sentiment_df.groupBy(["ASIN","sentiment"]).agg(concat_ws(" + ", collect_list( sentiment_df['review'])).alias("review_text"))
review_df.cache()

Out[7]: DataFrame[ASIN: string, sentiment: string, review_text: string]

In [0]:
review_df.show(5, truncate=400, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ASIN        | B00UN1XIN2                                                                                                                                                                                                                                                                                                                                                                                                       
 sentiment   | positive                                                                                                                                               

In [0]:
summarizer = pipeline("summarization")
def summarize_text(text):
    if len(text) < 1024:
        return summarizer(text, max_length=100, min_length=1, do_sample=True)[0]['summary_text']
    else:
        return summarize_text(text[0:int(len(text)/2)]) + summarize_text(text[int(len(text)/2)+1 : ])


# Summarize the review text
summarized_text_df = review_df.withColumn("summary_text", udf(summarize_text)(col("review_text")))
summarized_text_df.cache()
summarized_text_df.show(10, truncate=300, vertical=True)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ASIN         | B00UN1XIN2                                                                                                                                                                                                                                                                                                   
 sentiment    | positive                                                                                                                                                                                                                                                                                                     
 review_text  | Quality & super cute : Adorabl

In [0]:
positive_df = summarized_text_df.filter(col("sentiment")=='positive')
positive_df.show(1)
negative_df = summarized_text_df.filter(col("sentiment")=='negative')
negative_df.show(1)
neutral_df = summarized_text_df.filter(col("sentiment")=='neutral')
neutral_df.show(1)

+----------+---------+--------------------+--------------------+
|      ASIN|sentiment|         review_text|        summary_text|
+----------+---------+--------------------+--------------------+
|B00UN1XIN2| positive|Quality & super c...| The only thing t...|
+----------+---------+--------------------+--------------------+
only showing top 1 row

+----------+---------+--------------------+--------------------+
|      ASIN|sentiment|         review_text|        summary_text|
+----------+---------+--------------------+--------------------+
|B00O40PBEC| negative|kind of disappoin...| This beanie is v...|
+----------+---------+--------------------+--------------------+
only showing top 1 row

+----------+---------+--------------------+--------------------+
|      ASIN|sentiment|         review_text|        summary_text|
+----------+---------+--------------------+--------------------+
|B01B40ORSU|  neutral|Five Stars : luv ...| Necklace was bus...|
+----------+---------+--------------------